In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D
%matplotlib inline

Using TensorFlow backend.


In [2]:
base_dir = "./data/"

In [3]:
def select_sample(data, n=1):
    np.random.shuffle(data)
    negatives = []
    positives = []
    straights = []
    for d in data:
        if d[1] > 0:
            positives.append(d)
        elif d[1] < 0:
            negatives.append(d)
        else:
            straights.append(d)
    return np.array(negatives[:n] + positives[:n] + straights[:n])

def get_model():
    ch, row, col = 3, 160, 320  # camera format

    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,
            input_shape=(row, col, ch),
            output_shape=(row, col, ch)))
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mse", metrics=['accuracy'])
    return model
    
def data_gen(data):
    while 1:
        np.random.shuffle(data)
        for d in data:
            img = plt.imread(base_dir + d[0])
            #res = cv2.resize(img,None,fx=1/2, fy=1/2, interpolation = cv2.INTER_AREA)
            res = np.expand_dims(img, axis=0)
            yield res, np.array([d[1]])

In [4]:
data = pd.read_csv("./data/driving_log.csv")[['center', 'steering']].values
sample = select_sample(data)

In [5]:
training_data = sample
epochs = 20
validation_data = sample


model = get_model()
model.fit_generator(data_gen(training_data), samples_per_epoch=training_data.shape[0],
                    nb_epoch=epochs, validation_data=data_gen(validation_data),
                    nb_val_samples=validation_data.shape[0])

Epoch 1/20
3/3 [==============================] - 0s - loss: 0.0664 - acc: 0.3333 - val_loss: 2.9935 - val_acc: 0.0000e+00
Epoch 2/20
3/3 [==============================] - 0s - loss: 2.4934 - acc: 0.0000e+00 - val_loss: 0.4585 - val_acc: 0.0000e+00
Epoch 3/20
3/3 [==============================] - 0s - loss: 0.5905 - acc: 0.0000e+00 - val_loss: 6.9590 - val_acc: 0.0000e+00
Epoch 4/20
3/3 [==============================] - 0s - loss: 4.4174 - acc: 0.0000e+00 - val_loss: 2.9987 - val_acc: 0.3333
Epoch 5/20
3/3 [==============================] - 0s - loss: 0.9444 - acc: 0.3333 - val_loss: 3.7580 - val_acc: 0.0000e+00
Epoch 6/20
3/3 [==============================] - 0s - loss: 12.7564 - acc: 0.0000e+00 - val_loss: 7.7975 - val_acc: 0.0000e+00
Epoch 7/20
3/3 [==============================] - 0s - loss: 4.4350 - acc: 0.0000e+00 - val_loss: 19.8759 - val_acc: 0.0000e+00
Epoch 8/20
3/3 [==============================] - 0s - loss: 10.5937 - acc: 0.0000e+00 - val_loss: 3.5674 - val_acc: 0.00